In [1]:
import sys 
sys.path.append('../')
import os
from deephive.environment.deephive_utils import *
from deephive.environment.utils import *
import numpy as np 
from deephive.exploration.gp_surrogate import GPSurrogateModule
from deephive.environment.utils import filter_points
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import Matern
from dotenv import load_dotenv
load_dotenv()
import matplotlib.pyplot as plt

Using device: cpu


In [2]:
# #Experiment 1: VARIANCE DRIVEN EXPLORERS AND EXPLOITERS
config_path = "../config/exp_config.json"
model_path = "../models/exploiting_model.pth"
mode = "test"
env, agent_policy = initialize(config_path, mode=mode, model_path=model_path)
config = parse_config(config_path)

Loaded policy from:  ../models/exploiting_model.pth


In [6]:
env.split_ratio = 0.3
obs, role = env.reset()
print(role)

Resetting the GP
[array([0., 1., 0., 0., 1., 0., 0., 0., 1., 0.]), array([0., 1., 0., 0., 1., 0., 0., 0., 1., 0.])]


In [ ]:
agent_policy.std_controller.decay_std()
std_obs = agent_policy.std_controller.get_all_std(role)
std_obs


In [ ]:
action, next_point =  get_informed_action(env, number_of_points=10)
obs, reward, done, info = env.step(action)

In [ ]:
render = Render(env)

In [ ]:
render.render_state()

In [ ]:
env.surrogate.plot_surrogate()

In [ ]:
env.surrogate.plot_checkpoints_state()

In [ ]:
class StdController:
    def __init__(self, num_agents, role_std={'explorer': 1.0, 'exploiter': 0.5}, decay_rate=0.99, min_std=0.01, max_std=0.5):
        self.num_agents = num_agents
        self.std = [role_std['explorer']] * num_agents  # Default std for all agents
        self.role_std = role_std
        self.decay_rate = decay_rate
        self.min_std = min_std
        self.max_std = max_std
        self.iteration_num = 0

    def update_roles(self, roles):
        # roles: list of 0s and 1s indicating the role of each agent
        for i in range(self.num_agents):
            self.std[i] = self.role_std['explorer'] if roles[i] == 0 else self.role_std['exploiter']

    def decay_std(self, iteration):
        # Optionally decay std based on the iteration number
        decay_factor = self.decay_rate ** iteration
        self.std = [max(self.min_std, min(s * decay_factor, self.max_std)) for s in self.std]

    def get_std(self, agent_id):
        self.iteration_num += 1
        # Get the current std for a specific agent
        return self.std[agent_id]
    
    def get_all_std(self, roles=None):
        # Get the current std for all agents
        if roles is not None:
            self.update_roles(roles)
        return self.std
    
    
    




In [5]:
0.9**20

0.12157665459056935

In [ ]:
# Example usage
std_controller = StdController(num_agents=10,
                               role_std={'explorer': 0.2, 'exploiter': 0.05},
                               decay_rate=0.8,
                               min_std=0.01,
                               max_std=0.5)
std_controller.update_roles([0, 1, 0, 1, 0, 1, 0, 1, 0, 1])
std_controller.decay_std(iteration=10)

for agent_id in range(10):
    print(f"Agent {agent_id} Std: {std_controller.get_std(agent_id)}")

In [ ]:
std_controller.get_all_std()